In [ ]:
import numpy as np
import pandas as pd
import os
import shutil
from PIL import Image
import torch
from tqdm import tqdm
from distutils.dir_util import copy_tree

C:\Users\Dan\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
WORKING_DIR = ''

# Train

In [ ]:
names = [
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        '3.20',
        None,#10
        None,
        '2.1',
        '2.4',
        None,
        None,
        None,
        '3.1',
        None,
        None,
        None,#20
        None,
        None,
        None,
        None,
        '1.25',
        None,
        None,
        None,
        None,
        None,#30
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,#40
        None,
        None,
    ]

In [ ]:
classes = list([elem for elem in names if elem is not None])
classes

['3.20', '2.1', '2.4', '3.1', '1.25']

In [ ]:
with open(os.path.join(WORKING_DIR, 'FullIJCNN2013/gt.txt'), 'r') as f:
    read = f.read()
if 'img_name;xmin;ymin;xmax;ymax;class_id' not in read:
    with open(os.path.join(WORKING_DIR, 'FullIJCNN2013/gt.txt'), 'w') as f:
        f.write('img_name;xmin;ymin;xmax;ymax;class_id\n' + read)

In [ ]:
df = pd.read_csv(os.path.join(WORKING_DIR, 'FullIJCNN2013/gt.txt'), sep=';')

In [ ]:
os.makedirs(os.path.join(WORKING_DIR, 'dataset/images/train'), exist_ok=True)
os.makedirs(os.path.join(WORKING_DIR, 'dataset/images/val'), exist_ok=True)
os.makedirs(os.path.join(WORKING_DIR, 'dataset/labels/train'), exist_ok=True)
os.makedirs(os.path.join(WORKING_DIR, 'dataset/labels/val'), exist_ok=True)

In [ ]:
n = 0
for i in tqdm(range(len(df))):
    img_name, x1, y1, x2, y2, class_id = df.iloc[i]

    class_id = int(class_id)
    if names[class_id] is None:
        continue
    class_id = classes.index(names[class_id])
    
    img = Image.open(os.path.join(WORKING_DIR, 'FullIJCNN2013', img_name))
    w_img, h_img = img.size
    w, h = x2 - x1, y2 - y1
    x, y = x1 + w / 2, y1 + h / 2
    w /= w_img
    h /= h_img
    x /= w_img
    y /= h_img
    
    adding = 'train' if n >= 3 else 'val'
    
    img.save(os.path.join(WORKING_DIR, 'dataset/images/', adding, img_name.replace('.ppm', '.png')))
    if os.path.exists(os.path.join(WORKING_DIR, 'dataset/labels/', adding, img_name.replace('.ppm', '.txt'))):
        with open(os.path.join(WORKING_DIR, 'dataset/labels/', adding, img_name.replace('.ppm', '.txt')), 'r') as f:
            read = f.read()
    else:
        read = ''
    with open(os.path.join(WORKING_DIR, 'dataset/labels/', adding, img_name.replace('.ppm', '.txt')), 'w') as f:
        f.write(f'{class_id} {x} {y} {w} {h}\n' + read)
    n += 1

100%|██████████████████████████████████████████████████████████████████████████████| 1213/1213 [02:10<00:00,  9.29it/s]


In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


In [ ]:
data = f'''train: ../dataset/images/train/ 
val:  ../dataset/images/val/

nc: {len(classes)}

names: {str(classes).replace('"', "'")}'''
with open('yolov5/data/dataset.yaml', 'w') as f:
    f.write(data)

In [ ]:
%cd yolov5
!python train.py --cfg yolov5l.yaml --batch 4 --epochs 20 --data dataset.yaml --weights yolov5l.pt --name sign_detect
%cd ..

# Reusing model

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')

Using cache found in C:\Users\Dan/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-8-30 Python-3.10.3 torch-1.12.1+cu113 CUDA:0 (NVIDIA GeForce GTX 1080, 8192MiB)

Fusing layers... 
YOLOv5l summary: 267 layers, 46129818 parameters, 0 gradients, 107.7 GFLOPs
Adding AutoShape... 


In [ ]:
model.conf = 0.6

In [ ]:
sings = ['3.24', '1.16', '5.15.5', '5.19.1', '5.19.2', '1.20.1', '8.23',
'2.1', '4.2.1', '8.22.1', '6.16', '1.22', '1.2', '5.16', '3.27',
'6.10.1', '8.2.4', '6.12', '5.15.2', '3.13', '3.1', '3.20', '3.12',
'7.14.2', '5.23.1', '2.4', '5.6', '4.2.3', '8.22.3', '5.15.1',
'7.3', '3', '2.3.1', '3.11', '6.13', '5.15.4', '8.2.1', '1.34.3',
'8.2.2', '5.15.3', '1.17', '4.1.1', '4.1.4', '3.25', '1.20.2',
'8.22.2', '6.9.2', '3.2', '5.5', '5.15.7', '7.12', '8.2.3',
'5.24.1', '1.25', '3.28', '5.9.1', '5.15.6', '8.1.1', '1.10',
'6.11', '3.4', '6.10', '6.9.1', '8.2.5', '5.15', '4.8.2', '8.22',
'5.21', '5.18']

In [ ]:
classes = ['3.20', '2.1', '2.4', '3.1', '1.25']

In [ ]:
train = pd.read_csv(os.path.join(WORKING_DIR, 'train.csv'))

In [ ]:
os.makedirs(os.path.join(WORKING_DIR, 'dataset2/images/train'), exist_ok=True)
os.makedirs(os.path.join(WORKING_DIR, 'dataset2/images/val'), exist_ok=True)
os.makedirs(os.path.join(WORKING_DIR, 'dataset2/labels/train'), exist_ok=True)
os.makedirs(os.path.join(WORKING_DIR, 'dataset2/labels/val'), exist_ok=True)

In [ ]:
n = 0
for i in tqdm(range(len(train))):
    img_id, img_name, *signs = train.iloc[i]
    img = Image.open(os.path.join(WORKING_DIR, 'train', img_name))
    w_img, h_img = img.size
    preds = model(img).xyxy[0].tolist()
    if len(preds) == 0:
        continue
    correct_preds = []
    for x1, y1, x2, y2, conf, class_id in preds:
        class_id = int(class_id)
        class_name = classes[class_id]
        if class_name not in sings:
            continue
        if sings.index(class_name) + 1 not in signs:
            continue
        w, h = x2 - x1, y2 - y1
        x, y = x1 + w / 2, y1 + h / 2
        w /= w_img
        h /= h_img
        x /= w_img
        y /= h_img
        correct_preds.append(f'{class_id} {x} {y} {w} {h}')
    correct_preds = correct_preds[:8]
    adding = 'train' if n >= 3 else 'val'
    shutil.copy(os.path.join(WORKING_DIR, 'train', img_name), os.path.join(WORKING_DIR, 'dataset2/images', adding, img_name))
    with open(os.path.join(WORKING_DIR, 'dataset2/labels/', adding, img_name.replace('.jpg', '.txt')), 'w') as f:
        f.write('\n'.join(correct_preds))
    n += 1

100%|████████████████████████████████████████████████████████████████████████████████| 778/778 [00:32<00:00, 23.62it/s]


In [ ]:
data = f'''train: ../dataset2/images/train/ 
val:  ../dataset2/images/val/

nc: {len(classes)}

names: {str(classes).replace('"', "'")}'''
with open('yolov5/data/dataset2.yaml', 'w') as f:
    f.write(data)

In [ ]:
%cd yolov5
!python train.py --cfg yolov5l.yaml --batch 4 --epochs 20 --data dataset2.yaml --weights yolov5l.pt --name sign_detect2
%cd ..

In [ ]:
os.makedirs(os.path.join(WORKING_DIR, 'dataset3'), exist_ok=True)

In [ ]:
copy_tree(os.path.join(WORKING_DIR, 'dataset/'), os.path.join(WORKING_DIR, 'dataset3/'))

['dataset3/images\\train\\00007.png',
 'dataset3/images\\train\\00008.png',
 'dataset3/images\\train\\00010.png',
 'dataset3/images\\train\\00015.png',
 'dataset3/images\\train\\00016.png',
 'dataset3/images\\train\\00019.png',
 'dataset3/images\\train\\00024.png',
 'dataset3/images\\train\\00025.png',
 'dataset3/images\\train\\00028.png',
 'dataset3/images\\train\\00031.png',
 'dataset3/images\\train\\00032.png',
 'dataset3/images\\train\\00035.png',
 'dataset3/images\\train\\00036.png',
 'dataset3/images\\train\\00038.png',
 'dataset3/images\\train\\00039.png',
 'dataset3/images\\train\\00040.png',
 'dataset3/images\\train\\00043.png',
 'dataset3/images\\train\\00044.png',
 'dataset3/images\\train\\00046.png',
 'dataset3/images\\train\\00048.png',
 'dataset3/images\\train\\00049.png',
 'dataset3/images\\train\\00050.png',
 'dataset3/images\\train\\00051.png',
 'dataset3/images\\train\\00052.png',
 'dataset3/images\\train\\00054.png',
 'dataset3/images\\train\\00055.png',
 'dataset3/i

In [ ]:
copy_tree(os.path.join(WORKING_DIR, 'dataset2/'), os.path.join(WORKING_DIR, 'dataset3/'))

['dataset3/images\\train\\3-avi-frame672_jpg.rf.e995171158674d218e95be87ea251034.jpg',
 'dataset3/images\\train\\3-avi-frame672_jpg.rf.f437fd2f5a8f488c986672fc60e42676.jpg',
 'dataset3/images\\train\\3-avi-frame673_jpg.rf.3950bcb225c881e7ae554216b7a18c0b.jpg',
 'dataset3/images\\train\\3-avi-frame674_jpg.rf.36e6b1a4c3dd49e77b7a57c5a4c1da19.jpg',
 'dataset3/images\\train\\3-avi-frame676_jpg.rf.f1e172c4dc014869f1cc808bf02a4958.jpg',
 'dataset3/images\\train\\3-avi-frame677_jpg.rf.7fa00aba881e9d2ce4336f32fe89c50e.jpg',
 'dataset3/images\\train\\3-avi-frame677_jpg.rf.e9334199bc90928c3004006d5a266646.jpg',
 'dataset3/images\\train\\3-avi-frame678_jpg.rf.48c3a83b49eedd47a395845c93a87e1e.jpg',
 'dataset3/images\\train\\3-avi-frame679_jpg.rf.d94d38acccb11156b5c5044055c7c514.jpg',
 'dataset3/images\\train\\3-avi-frame680_jpg.rf.d42e9e3dacf2bd9d2f8d951bb00f5a34.jpg',
 'dataset3/images\\train\\3-avi-frame681_jpg.rf.31842d3de8f346ef989ee08488999832.jpg',
 'dataset3/images\\train\\5-avi-frame228_jp

In [ ]:
data = f'''train: ../dataset3/images/train/ 
val:  ../dataset3/images/val/

nc: {len(classes)}

names: {str(classes).replace('"', "'")}'''
with open('yolov5/data/dataset3.yaml', 'w') as f:
    f.write(data)

In [ ]:
%cd yolov5
!python train.py --cfg yolov5l.yaml --batch 4 --epochs 40 --data dataset3.yaml --weights yolov5l.pt --name sign_detect2
%cd ..